In [1]:
from import_libraries.project_functions import *

with open('data/private_data.json', 'r') as f:
    private_data = json.load(f)


account = private_data["account"]
password = private_data["password"]
server = private_data["server"]

In [2]:
def connect_to_mt5():
    global account, password, server
    # установка соединения с MetaTrader 5
    if not mt5.initialize():
        print("initialize() failed")
        mt5.shutdown()
        return None
        
    authorized = mt5.login(account, password=password, server=server)
    
    if authorized:
        print(f"Connected to account {account}")
        return True
    else:
        print("Failed to connect. Error code:", mt5.last_error())
        mt5.shutdown()
        return False

def get_historical_data(timeframe=mt5.TIMEFRAME_D1, symbol="GER30", start='2012-01-01'):
    """
    mt5.TIMEFRAME_M5
    mt5.TIMEFRAME_M1
    """

    if not connect_to_mt5():
        return None

    # получение исторических данных
    start_time = pd.Timestamp(start)  # начальная дата
   
    count = 22734   # Количество свечей, которые вы хотите получить
    rates = mt5.copy_rates_from(symbol, timeframe, start_time, count)

    # проверка успешного получения данных
    if rates is not None:
        # преобразование данных в DataFrame pandas
        rates_frame = pd.DataFrame(rates)
        rates_frame['time'] = pd.to_datetime(rates_frame['time'], unit='s')
        # закрытие соединения с MetaTrader 5
        mt5.shutdown()
        return rates_frame
    else:
        print("Ошибка при получении данных.")
        mt5.shutdown()
        return None


# Пример использования функции
data_d1 = get_historical_data()
data_m5 = get_historical_data(timeframe=mt5.TIMEFRAME_M5, symbol="GER30", start='2022-01-01')

Connected to account 61948068
Connected to account 61948068


In [3]:
data_m5

,time,open,high,low,close,tick_volume,spread,real_volume
0,2021-07-12 17:35:00,15773.0,15780.0,15770.0,15776.0,330,0,0
1,2021-07-12 17:40:00,15775.0,15777.0,15765.0,15769.0,258,0,0
2,2021-07-12 17:45:00,15770.0,15778.0,15770.0,15773.0,164,10,0
3,2021-07-12 17:50:00,15773.0,15779.0,15773.0,15777.0,125,0,0
4,2021-07-12 17:55:00,15777.0,15782.0,15777.0,15781.0,117,10,0
...,...,...,...,...,...,...,...,...
20544,2021-12-30 21:35:00,15857.0,15860.0,15853.0,15857.0,190,10,0
20545,2021-12-30 21:40:00,15856.0,15857.0,15850.0,15850.0,224,0,0
20546,2021-12-30 21:45:00,15850.0,15853.0,15843.0,15847.0,242,10,0
20547,2021-12-30 21:50:00,15847.0,15851.0,15834.0,15841.0,579,10,0
